In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [39]:

class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(BinaryClassifier, self).__init__()
        layers = []
        #add fully connected layers with the input dim dividing by two each time the input shape
        output_dim = max(input_dim//(2), 1)
        for i in range(hidden_layers):
            i += 1
            layers.append(nn.Linear(input_dim,output_dim))
            layers.append(nn.ReLU())
            input_dim = output_dim
            output_dim = max(input_dim//(2*i), 1)
        #add final layer with sigmiod output and final linear layer
        layers.append(nn.Linear(input_dim,1))
        layers.append(nn.Sigmoid())

        #create a Seqeuntial implementation, no additional layers but provides one line to run through all layers
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        #apply the neural net
        return self.network(x)

In [41]:
#input shape is not correct
input_shape = 30
hidden_layers = 5
model_1 = BinaryClassifier(input_shape, hidden_layers)

In [43]:
optimizer = optim.Adam(model_1.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

In [45]:
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

In [47]:

def dataset_to_numpy(dataset: TensorDataset):
    X, y = dataset.tensors
    # Ensure on CPU and detached from computation graph
    X_np = X.detach().numpy()
    y_np = y.detach().numpy()
    return X_np, y_np


In [65]:
def load_data():
    # Read data
    df = pd.read_csv('cleaned_cancer_data.csv', encoding='latin-1')

    # Separate features and labels
    X = df.drop(columns=['Diagnosis_encoded', 'Diagnosis']).to_numpy()
    y = df['Diagnosis_encoded'].to_numpy()
    return X, y

In [67]:
#import whatever data loader function is needed
#should output train dataloader and test dataloader  as the full train and test datasets
X_np, y_np = load_data()
print(X_np.shape, y_np.shape)

(569, 30) (569,)


In [ ]:
#turn train data into epochs

In [ ]:
# Training loop
# def train(loss_fn,optimizer, data, model):
#     num_epochs = 10
#     epochs_loss = []
#     epochs_acr = []
#     for epoch in range(num_epochs):
#         model.train()
#         epoch_loss = 0.0
#         correct = 0
#         total = 0
        
#         for batch_X, batch_y in data:
#             optimizer.zero_grad()
            
#             outputs = model(batch_X)            # forward
#             loss = loss_fn(outputs, batch_y)  # compute loss
#             loss.backward()                     # backprop
#             optimizer.step()                    # update weights
            
#             epoch_loss += loss.item() * batch_X.size(0)
            
#             # accuracy calculation
#             preds = (outputs > 0.5).float()
#             correct += (preds == batch_y).sum().item()
#             total += batch_y.size(0)
        
#         avg_loss = epoch_loss / total
#         accuracy = correct / total
#         epochs_loss.append(avg_loss)
#         epochs_acr.append(accuracy)
#         print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} Acc: {accuracy:.4f}")
#     return model, epochs_loss, epochs_acr

In [ ]:
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [20]:
def test(model, data, criterion):
    model.eval()  # evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_X, batch_y in data:
            
            logits = model(batch_X)                # raw outputs
            loss = criterion(logits, batch_y)
            
            test_loss += loss.item() * batch_X.size(0)
            
            # convert logits -> probabilities -> predictions
            preds = torch.sigmoid(logits) > 0.5
            correct += (preds.float() == batch_y).sum().item()
            total += batch_y.size(0)
    
    avg_loss = test_loss / total
    accuracy = correct / total
    print(f"Test Loss: {avg_loss:.4f} | Test Acc: {accuracy:.4f}")
    return avg_loss, accuracy
    
def train_with_stratified_kfold(X,y, input_shape, hidden_layers, k_folds=5, num_epochs=10):
    #X and y should be numpy arrays
    skf = StratifiedKFold(n_splits = 4,shuffle=True)
    all_fold_results = []
    for i, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        
        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)

        train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        cur_model = model()
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        for epoch in range(num_epochs):
            model.train()
            epoch_loss = 0.0
            correct = 0
            total = 0
            for batch_X , batch_Y in train_loader:
                optimizer.zero_grad()
                outputs = cur_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backwards()
                optimizer.step()
                
                epoch_loss += loss.item() * batch_X.size(0)
                preds = (outputs >= 0.5).float()
                correct += (preds == batch_y).sum().item()
                total += batch_y.size(0)

            train_acc = correct / total
            train_loss = epoch_loss / total

            avg_loss, accuracy = test(cur_model, val_loader, criterion)

            all_fold_results.append((avg_loss,accurancy)
    
    return all_fold_results

IndentationError: expected an indented block after 'for' statement on line 36 (334902035.py, line 38)

In [ ]:
def test(model, data, criterion):
    model.eval()  # evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_X, batch_y in data:
            
            logits = model(batch_X)                # raw outputs
            loss = criterion(logits, batch_y)
            
            test_loss += loss.item() * batch_X.size(0)
            
            # convert logits -> probabilities -> predictions
            preds = torch.sigmoid(logits) > 0.5
            correct += (preds.float() == batch_y).sum().item()
            total += batch_y.size(0)
    
    avg_loss = test_loss / total
    accuracy = correct / total
    print(f"Test Loss: {avg_loss:.4f} | Test Acc: {accuracy:.4f}")
    return avg_loss, accuracy

In [ ]:
#training model
trained_model_1,epochs_loss,epochs_arc = train(loss_fn,optimizer, train_epoch_data, model_1)

In [ ]:
#testing model
avg_loss, accuracy = test(trained_model, train_data, loss_fn)
print(f'testing loss {avg_loss}, model accuracy {accuracy}')

In [ ]:
plt.plot(epochs_loss)
plt.ylabel('training loss')
plt.xlabel('epochs')
plt.show()

In [ ]:
# # Compatibility check
# print("=== COMPATIBILITY CHECK ===")
# print(f"Input shape: {input_shape}")
# print(f"Hidden layers: {hidden_layers}")
# print(f"Train data type: {type(train_data)}")

# # Test model creation
# test_model = BinaryClassifier(input_shape, hidden_layers)
# print(f"Model created successfully: {test_model}")

# # Test with one batch
# for batch_X, batch_y in train_data:
#     print(f"Batch X shape: {batch_X.shape}")
#     print(f"Batch y shape: {batch_y.shape}")
#     print(f"Batch y values: {batch_y.unique()}")
    
#     # Test forward pass
#     test_output = test_model(batch_X)
#     print(f"Model output shape: {test_output.shape}")
#     print(f"Model output range: {test_output.min().item():.4f} to {test_output.max().item():.4f}")
#     break

# print("✅ All compatibility checks passed!")